In [1]:
import urllib.request 
from urllib.parse import quote 
from bs4 import BeautifulSoup
import json
import re
import time

In [2]:
#Source: https://codezup.com/web-scraping-word-meaning-dictionary-python-beautifulsoup/

In [3]:
with open('./wordList', 'r') as infile:
    wordList = infile.read()

In [4]:
wordList = wordList.split()

In [5]:
wordList

['một',
 'và',
 'của',
 'là',
 'có',
 'không',
 'tôi',
 'người',
 'những',
 'đã',
 'ông',
 'cho',
 'như',
 'ta',
 'trong']

In [6]:
#The link format for vdict Viet-Fran is: https://vdict.com/m%E1%BB%99t,4,0,0.html

In [7]:
#Converting IRI to ASCII:
#https://stackoverflow.com/questions/4389572/how-to-fetch-a-non-ascii-url-with-python-urlopen

In [8]:
def findTranslation(word):
    url = "https://vdict.com/" + quote(word) + ",4,0,0.html" #resolving the IRI issue
    
    try: #make sure the word is in the dictionary
        source = urllib.request.urlopen(url)
    except urllib.HTTPError:
        print("Word not found " + word)
        return "N/A"
    
    soup = BeautifulSoup(source, 'lxml')

    translations = {}

    #Getting the first translation
    curTag = soup.find("ul", class_= "list1")

    #storing the number of translations a word has
    numTrans = 1

    #continue while we still have more translations
    while(curTag != '\n'): 
        temp = {} #temp dict to store each translation of a word

        #getting the raw translation
        temp['translation'] = re.split('[,;]', curTag.find('b').string)

        #retrieving the examples of each of the translations
        try:
            examples = curTag.findAll("ul", class_= "list2")
            temp2 = {} #another dict to store each examples of a translation

            numExamples = len(list(examples))
            temp2['nums'] = numExamples

            for j in range(0, numExamples):
                temp2['context ' + str(j+1)] = examples[j].find(class_ = "example-original").string

                #retrieving the usage of each examples
                try: 
                    tempString = str(examples[j].find("li")) #extracting the usage
                    temp2['usage ' + str(j+1)] = tempString[tempString.find("<br/>")+5:-5]
                except:
                    temp2['usage ' + str(j+1)] = "N/A"

            temp['examples'] = temp2
        except:
            temp['examples'] = "N/A"

        translations[numTrans] = temp #adding the translation to the dictionary
        
        #go to next tag              
        numTrans += 1
        curTag = curTag.next_sibling
    
    translations["nums"] = numTrans - 1
        
    return translations

In [9]:
url = "https://vdict.com/" + quote("có") + ",4,0,0.html" #resolving the IRI issue

try:
    source = urllib.request.urlopen(url)
except:
    print( "Link broken")
soup = BeautifulSoup(source, 'lxml')

In [10]:
soup.find("ul", class_= "list1").next_sibling.next_sibling.next_sibling.next_sibling.next_sibling.next_sibling.next_sibling.next_sibling.next_sibling.next_sibling

'\n'

In [11]:
containers = soup.findAll("ul", class_= "list1")
containers

[<ul class="list1"><li><b>avoir; posséder</b><ul class="list2"><li><span class="example-original">Tôi có nói câu đó </span><br/>j'ai dit cette phrase</li></ul><ul class="list2"><li><span class="example-original">Có tiền </span><br/>avoir de l'argent</li></ul><ul class="list2"><li><span class="example-original">Có một cái nhà </span><br/>posséder une maison</li></ul></li></ul>,
 <ul class="list1"><li><b>exister; se trouver</b><ul class="list2"><li><span class="example-original">Cái tục đó còn đó </span><br/>cet usage existe encore</li></ul><ul class="list2"><li><span class="example-original">Loài chim đó không có ở Việt Nam </span><br/>cette espèce d'oiseau n'existe pas au Viet-Nam</li></ul><ul class="list2"><li><span class="example-original">Tên anh không có trong danh sách </span><br/>ton nom ne se trouve pas sur la liste</li></ul></li></ul>,
 <ul class="list1"><li><b>disposer de</b><ul class="list2"><li><span class="example-original">ông ấy có một cái xe </span><br/>il dispose d'une 

In [12]:
meanings = [item.find('b').string for item in containers[0:4]] #word translation
meanings

['avoir; posséder', 'exister; se trouver', 'disposer de', 'être présent']

In [13]:
examples = containers[0].findAll("ul", class_= "list2")
temp = str(examples[1].find("li"))
temp[temp.find("<br/>")+5:-5]

"avoir de l'argent"

In [14]:
dictionary = {}

In [15]:
start_time = time.time()
for word in wordList:
    dictionary[word] = findTranslation(word)
    time.sleep(1) #wait for 1 sec between each query
print("--- %s seconds ---" % (time.time() - start_time))

--- 43.75471472740173 seconds ---


In [16]:
dictionary

{'một': {1: {'translation': ['un'],
   'examples': {'nums': 3,
    'context 1': 'Một người ',
    'usage 1': 'un homme;',
    'context 2': 'Một nhà ',
    'usage 2': 'une maison;',
    'context 3': 'Nước Việt Nam là một ',
    'usage 3': 'le Viêtnam est un'}},
  2: {'translation': ['unique'],
   'examples': {'nums': 2,
    'context 1': 'Con một ',
    'usage 1': 'fils unique;',
    'context 2': 'Một màu ',
    'usage 2': "d'une seule couleur; unicolore"}},
  3: {'translation': ['pour une seule personne', ''],
   'examples': {'nums': 16,
    'context 1': 'Màn dành cho một người ngủ ',
    'usage 1': 'moustiquaire pour une seule personne',
    'context 2': 'chỉ là một ',
    'usage 2': "c'est tout un;",
    'context 3': 'một chín một mười ',
    'usage 3': 'équivalent',
    'context 4': 'một chốn đôi nơi ',
    'usage 4': 'ménage à deux foyers; famille à deux foyers;',
    'context 5': 'một chữ cắn đôi cũng không biết ',
    'usage 5': 'illettré;',
    'context 6': 'một cổ hai tròng ',
 

In [17]:
with open('vdict VI-FR.json', 'w') as outfile:
    json.dump(dictionary, outfile)

In [18]:
findTranslation("")

{1: {'translation': ['avoir', ' posséder'],
  'examples': {'nums': 3,
   'context 1': 'Tôi có nói câu đó ',
   'usage 1': "j'ai dit cette phrase",
   'context 2': 'Có tiền ',
   'usage 2': "avoir de l'argent",
   'context 3': 'Có một cái nhà ',
   'usage 3': 'posséder une maison'}},
 2: {'translation': ['exister', ' se trouver'],
  'examples': {'nums': 3,
   'context 1': 'Cái tục đó còn đó ',
   'usage 1': 'cet usage existe encore',
   'context 2': 'Loài chim đó không có ở Việt Nam ',
   'usage 2': "cette espèce d'oiseau n'existe pas au Viet-Nam",
   'context 3': 'Tên anh không có trong danh sách ',
   'usage 3': 'ton nom ne se trouve pas sur la liste'}},
 3: {'translation': ['disposer de'],
  'examples': {'nums': 1,
   'context 1': 'ông ấy có một cái xe ',
   'usage 1': "il dispose d'une voiture"}},
 4: {'translation': ['être présent'],
  'examples': {'nums': 1,
   'context 1': 'Một chất khí có rất ít trong không khí ',
   'usage 1': "un gaz qui est présent dans l'air en proportion in

In [19]:
findTranslation("là")

{1: {'translation': ['repasser'],
  'examples': {'nums': 1,
   'context 1': 'Bàn là ',
   'usage 1': 'fer à repasser'}},
 2: {'translation': ['raser'],
  'examples': {'nums': 1,
   'context 1': 'Máy bay là mặt đất ',
   'usage 1': 'avion qui rase le sol'}},
 3: {'translation': ['être', ' constituer'],
  'examples': {'nums': 2,
   'context 1': 'Chúng ta là người Việt Nam ',
   'usage 1': 'nous sommes Vietnamiens',
   'context 2': 'Hành động đó là một tội ',
   'usage 2': 'cette action est un délit'}},
 4: {'translation': ['égaler', ' faire'],
  'examples': {'nums': 1,
   'context 1': 'Hai với hai là bốn ',
   'usage 1': 'deux et deux font quatre'}},
 5: {'translation': ['que', ' comme', ' en tant que'],
  'examples': {'nums': 3,
   'context 1': 'Tôi nghĩ là anh có lí ',
   'usage 1': 'je pense que vous avez raison',
   'context 2': 'Cô ấy được chọn là thư kí ',
   'usage 2': 'elle a été choisie comme secrétaire',
   'context 3': 'Là giám đốc , anh ấy phải quản lí tốt nhà máy ấy ',
   'u